In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['TR']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 1]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('thyroid hormone receptor', 399),
 ('tricuspid regurgitation', 137),
 ('thioredoxin reductase', 96),
 ('repetition time', 67),
 ('terminal repeat', 55),
 ('th receptor', 48),
 ('t3 receptor', 46),
 ('thyroid receptor', 43),
 ('trypanothione reductase', 40),
 ('t cell receptor', 36),
 ('telomerase rna', 33),
 ('tandem repeat', 30),
 ('transferrin receptor', 19),
 ('taste reactivity', 17),
 ('training', 16),
 ('target replacement', 16),
 ('tricuspid valve regurgitation', 16),
 ('transpiration rate', 15),
 ('transcription rate', 12),
 ('texas red', 11),
 ('telerehabilitation', 10),
 ('transradial', 9),
 ('thearubigins', 9),
 ('tramadol', 8),
 ('transitional', 8),
 ('tranilast', 8),
 ('treatment resistant', 8),
 ('trastuzumab resistant', 8),
 ('thioredoxin', 8),
 ('tiazofurin', 8),
 ('thrombin receptor', 8),
 ('time restricted', 7),
 ('time reverse', 7),
 ('transrepression', 7),
 ('transcription regulation', 7),
 ('translucent', 6),
 ('treatment', 6),
 ('trochanter', 6),
 ('taxonomic ric

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-10-02 03:50:43] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [6]:
result = [grounding_map, names, pos_labels]

In [7]:
result

[{'thyroid hormone receptor': 'FPLX:THR',
  'tricuspid regurgitation': 'MESH:D014262',
  'repetition time': 'MESH:D013995',
  'terminal repeat': 'MESH:D020079',
  'th receptor': 'GO:GO:0038023',
  'thyroid receptor': 'GO:GO:0038023',
  't cell receptor': 'MESH:D011948',
  'telomerase rna': 'CHEBI:CHEBI:33697',
  'tandem repeat': 'MESH:D020080',
  'transferrin receptor': 'HGNC:11763',
  'tricuspid valve regurgitation': 'MESH:D014262',
  'texas red': 'MESH:C034657',
  'telerehabilitation': 'MESH:D000069350',
  'tramadol': 'CHEBI:CHEBI:9648',
  'tranilast': 'CHEBI:CHEBI:77572',
  'thioredoxin': 'FPLX:TXN',
  'tiazofurin': 'MESH:C033706',
  'thrombin receptor': 'MESH:D018179',
  'treatment': 'MESH:D013812',
  'treadmill running': 'MESH:D012420',
  'regulatory t': 'CHEBI:CHEBI:16857',
  'temperature': 'MESH:D013696',
  'thioridazine': 'CHEBI:CHEBI:9566',
  't3 and its receptor': 'GO:GO:0038023',
  'true local recurrence': 'MESH:D012008',
  'receptor for thyroid hormone': 'CHEBI:CHEBI:60311'

In [9]:
grounding_map, names, pos_labels = [{'receptor for thyroid hormone': 'FPLX:THR',
  'regulatory t': 'ungrounded',
  'repetition time': 'ungrounded',
  'rna stem loop': 'ungrounded',
  'rna template': 'ungrounded',
  't 3 receptor': 'FPLX:THR',
  't cell receptor': 'MESH:D011948',
  't3 and its receptor': 'FPLX:THR',
  't3 receptor': 'FPLX:THR',
  'talin rod': 'ungrounded',
  'tamoxifen resistant': 'ungrounded',
  'tandem and ring': 'ungrounded',
  'tandem repeat': 'MESH:D020080',
  'target replacement': 'ungrounded',
  'target resequencing': 'ungrounded',
  'taste reactivity': 'ungrounded',
  'taxonomic rich': 'ungrounded',
  'telerehabilitation': 'MESH:D000069350',
  'telomerase rna': 'MESH:C422952',
  'telomerase rna components': 'MESH:C422952',
  'temperature': 'ungrounded',
  'template region': 'ungrounded',
  'template repeat': 'ungrounded',
  'tendon reflex': 'ungrounded',
  'terc': 'ungrounded',
  'terminal redundancy': 'ungrounded',
  'terminal region': 'ungrounded',
  'terminal repeat': 'MESH:D020079',
  'texas red': 'CHEBI:CHEBI:51247',
  'th receptor': 'FPLX:THR',
  'thearubigins': 'ungrounded',
  'thioredoxin': 'FPLX:TXNRD',
  'thioredoxin reductase': 'FPLX:TXNRD',
  'thioridazine': 'CHEBI:CHEBI:9566',
  'thoracic restraint': 'ungrounded',
  'thrombin receptor': 'MESH:D018179',
  'thyroid hormone nuclear receptor': 'FPLX:THR',
  'thyroid hormone receptor': 'FPLX:THR',
  'thyroid receptor': 'FPLX:THR',
  'tiazofurin': 'MESH:C033706',
  'time restricted': 'ungrounded',
  'time restricted feeding': 'ungrounded',
  'time reverse': 'ungrounded',
  'tomoregulin': 'ungrounded',
  'total recall': 'ungrounded',
  'training': 'ungrounded',
  'tramadol': 'CHEBI:CHEBI:9648',
  'tranilast': 'CHEBI:CHEBI:77572',
  'transcription rate': 'ungrounded',
  'transcription regulation': 'ungrounded',
  'transcription repression': 'ungrounded',
  'transferrin receptor': 'HGNC:11763',
  'transitional': 'ungrounded',
  'transitional region': 'ungrounded',
  'translational rate': 'ungrounded',
  'translational repression': 'ungrounded',
  'translucent': 'ungrounded',
  'transpiration rate': 'ungrounded',
  'transport rate': 'ungrounded',
  'transradial': 'ungrounded',
  'transrepression': 'ungrounded',
  'trap roots': 'ungrounded',
  'trastuzumab resistant': 'ungrounded',
  'traveling ratio': 'ungrounded',
  'tread rubber': 'ungrounded',
  'treadmill running': 'MESH:D012420',
  'treatment': 'MESH:D013812',
  'treatment resistant': 'ungrounded',
  'treatment response': 'ungrounded',
  'triceps': 'ungrounded',
  'tricuspid regurgitation': 'MESH:D014262',
  'tricuspid valve regurgitation': 'MESH:D014262',
  'triiodothyronine receptor': 'FPLX:THR',
  'triose reductone': 'PUBCHEM:3034155',
  'trisomy rescue': 'ungrounded',
  'trochanter': 'ungrounded',
  'true local recurrence': 'MESH:D012008',
  'true recurrence': 'MESH:D012008',
  'true simple': 'ungrounded',
  'trypanothione reductase': 'IP:IPR001864',
  'trypsin': 'CHEBI:CHEBI:9765',
  'trypsin resistant': 'ungrounded',
  'tubular reabsorption': 'ungrounded',
  'tumor response': 'ungrounded'},
 {'FPLX:THR': 'THR',
  'MESH:D011948': 'Receptors, Antigen, T-Cell',
  'MESH:D020080': 'Tandem Repeat Sequences',
  'MESH:D000069350': 'Telerehabilitation',
  'MESH:C422952': 'telomerase RNA',
  'MESH:D020079': 'Terminal Repeat Sequences',
  'CHEBI:CHEBI:51247': 'Texas red',
  'FPLX:TXNRD': 'TXNRD',
  'CHEBI:CHEBI:9566': 'thioridazine',
  'MESH:D018179': 'Receptors, Thrombin',
  'MESH:C033706': 'tiazofurin',
  'CHEBI:CHEBI:9648': 'tramadol',
  'CHEBI:CHEBI:77572': 'tranilast',
  'HGNC:11763': 'TFRC',
  'MESH:D012420': 'Running',
  'MESH:D013812': 'Therapeutics',
  'MESH:D014262': 'Tricuspid Valve Insufficiency',
  'PUBCHEM:3034155': '(E)-2,3-Dihydroxyprop-2-enal',
  'MESH:D012008': 'Recurrence',
  'IP:IPR001864': 'Trypanothione reductase',
  'CHEBI:CHEBI:9765': 'Trypsin'},
 ['FPLX:THR',
  'FPLX:TXNRD',
  'HGNC:11763',
  'IP:IPR001864',
  'MESH:C422952',
  'MESH:D011948',
  'MESH:D020079',
  'MESH:D020080']]

In [10]:
excluded_longforms = []

In [11]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [12]:
additional_entities = {}

In [13]:
unambiguous_agent_texts = {}

In [14]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [15]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [16]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [17]:
intersection1

[]

In [18]:
intersection2

[]

In [20]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=['RTCA', 'RTCD1', 'RPC', 'RTC1', 'RTC'])
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [ ]:
names.update(additional_entitie)

In [19]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-23 03:24:55] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-23 03:25:26] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9359647300373034 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [16]:
classifier.stats

{'label_distribution': {'FPLX:THR': 487,
  'FPLX:TXNRD': 74,
  'MESH:C033706': 7,
  'CHEBI:CHEBI:CHEBI:51247': 10,
  'MESH:D013696': 6,
  'ungrounded': 224,
  'HGNC:11763': 15,
  'PUBCHEM:3034155': 3,
  'CHEBI:CHEBI:9765': 2,
  'IP:IPR001864': 31,
  'MESH:D018179': 8,
  'MESH:D020079': 38,
  'MESH:D020080': 21,
  'MESH:D012420': 6,
  'MESH:D014262': 119,
  'MESH:D012008': 4,
  'MESH:D011948': 23,
  'MESH:C422952': 28,
  'CHEBI:CHEBI:77572': 5,
  'CHEBI:CHEBI:9648': 5,
  'CHEBI:CHEBI:9566': 2,
  'MESH:D013812': 3,
  'MESH:D000069350': 5},
 'f1': {'mean': 0.934295, 'std': 0.008374},
 'precision': {'mean': 0.954037, 'std': 0.005581},
 'recall': {'mean': 0.931621, 'std': 0.014477},
 'MESH:D000069350': {'f1': {'mean': 0.4, 'std': 0.489898},
  'pr': {'mean': 0.4, 'std': 0.489898},
  'rc': {'mean': 0.4, 'std': 0.489898}},
 'MESH:C422952': {'f1': {'mean': 0.874747, 'std': 0.110946},
  'pr': {'mean': 0.86, 'std': 0.071181},
  'rc': {'mean': 0.911111, 'std': 0.177778}},
 'MESH:C033706': {'f1': {

In [20]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [21]:
disamb.dump(model_name, results_path)

In [22]:
print(disamb.info())

Disambiguation model for TR

Produces the disambiguations:
	(E)-2,3-Dihydroxyprop-2-enal	PUBCHEM:3034155
	Receptors, Antigen, T-Cell*	MESH:D011948
	Receptors, Thrombin	MESH:D018179
	Recurrence	MESH:D012008
	Running	MESH:D012420
	TFRC*	HGNC:11763
	THR*	FPLX:THR
	TXNRD*	FPLX:TXNRD
	Tandem Repeat Sequences*	MESH:D020080
	Telerehabilitation	MESH:D000069350
	Terminal Repeat Sequences*	MESH:D020079
	Texas red	CHEBI:CHEBI:51247
	Therapeutics	MESH:D013812
	Tricuspid Valve Insufficiency	MESH:D014262
	Trypanothione reductase*	IP:IPR001864
	Trypsin	CHEBI:CHEBI:9765
	telomerase RNA*	MESH:C422952
	thioridazine	CHEBI:CHEBI:9566
	tiazofurin	MESH:C033706
	tramadol	CHEBI:CHEBI:9648
	tranilast	CHEBI:CHEBI:77572

Class level metrics:
--------------------
Grounding                    	Count	F1     
                          THR*	487	0.96801
                   Ungrounded	230	0.85312
Tricuspid Valve Insufficiency	119	0.98741
                        TXNRD*	 74	0.93287
    Terminal Repeat Sequences*	 38	0.957

In [23]:
model_to_s3(disamb)